# SHscore and evolutionary distance

Here you can find the code to reproduce the results shown in the section **Experiment 2: SHscore and evolutionary distance** of the paper. SHscores and MRCA distances have been precomputed because the computation, here, would take too much time. 

You may refer to the other notebooks in the current repository or to the documentation at the official [PhyliCS repository](https://github.com/bioinformatics-polito/PhyliCS) to learn how to compute the SHscore.

## Experiment 1: SHscore VS MRCA distance